In [ ]:
from pathlib import Path
from genetic_algorithm import genetic_algorithm
from genetic_algorithm.selection import TournamentSelection
from mwcnf_parser import parse_mwcnf
from solver import MwcnfGenerator
from solver.fitness import satisfied_clause_count, weights
from solver.mwcnf_individual import MwcnfIndividual
from weighted_formula.weighted_formula import WeightedCnf

from io import StringIO
import matplotlib.pyplot as plt


In [ ]:
def evaluate_hyperparameters(formula_file: str | Path,
                             population_size: int,
                             generation_cnt: int,
                             tournament_size: float,
                             crossover_probability: float,
                             mutation_rate: float,
                             elitism: int,
                             print_plot: bool = True):


    debug_stream = StringIO()

    formula_path = Path(formula_file)
    formula = parse_mwcnf(formula_path)
    solution, i = genetic_algorithm(MwcnfGenerator(formula),
                                    population_size=population_size,
                                    number_of_generations=generation_cnt,
                                    selection=TournamentSelection(tournament_size),
                                    crossover_probability=crossover_probability,
                                    mutation_rate=mutation_rate,
                                    elitism=elitism,
                                    terminate_on_stagnation_in_x_generations=300,
                                    debug_stream=debug_stream if print_plot else None)

    if print_plot:
        orders = []
        bests = []
        means = []
        medians = []
        worsts = []
        for line in debug_stream.getvalue().splitlines():
            order, best, mean_, median_, worst = line.split(sep=',')
            orders.append(abs(int(order)))
            bests.append(abs(float(best)))
            means.append(abs(float(mean_)))
            medians.append(abs(float(median_)))
            worsts.append(abs(float(worst)))
        
        plt.plot(orders, bests, orders, means, orders, medians, orders, worsts)
        plt.show()

    if isinstance(solution, MwcnfIndividual):
        print(f"Number of generations: {i}")
        print(f"Number satisfied: {-satisfied_clause_count(formula, solution.config)}")
        print(f"{formula_path.name} {-weights(formula, solution.config)} {solution.config.get_evaluation()} 0")

evaluate_hyperparameters("wuf100-430/wuf100-430-Q/wuf100-01.mwcnf",
                         population_size=500,
                         generation_cnt=3000,
                         tournament_size=1.75,
                         crossover_probability=0.9,
                         mutation_rate=0.01,
                         elitism=1,
                         print_plot=True)

In [ ]:
from itertools import product


instances = [
    "wuf75-325/wuf75-325-Q/wuf75-01.mwcnf",
    "wuf50-218/wuf50-218-Q/wuf50-01.mwcnf",
    "wuf36-157/wruf36-157-Q/wruf36-157-1.mwcnf",    
    "wuf20-91/wuf20-91-Q/wuf20-01.mwcnf",
]
population_sizes = [
    1000,
    500,
    200,
    100
]
generation_counts = [
    500,
    200,
    100,
    50,
]
tournament_sizes = [
    5,
    3,
    2.5,
    2,
    1.75,
]
crossovers = [
    1,
    0.9,
    0.7,
    0.5,
    0.3,
    0.1,
    0,
]
mutation_rates = [
    0.02,
    0.015,
    0.01,
    0.005,
]
stagnations = [
    0.5
]

combinations = product(instances, population_sizes, generation_counts, tournament_sizes, crossovers, mutation_rates, stagnations)


In [ ]:
out_folder = Path("out")
for instance, pop_size, gen_cnt, tour_size, cross, mut, stag in combinations:
    print(f"{instance=},{pop_size=},{gen_cnt=},{tour_size=},{cross=},{mut=},{stag=}")
    debug_stream = StringIO()
    formula_path = Path(instance)
    formula = parse_mwcnf(formula_path)
    for j in range(100):
        solution, i = genetic_algorithm(MwcnfGenerator(formula),
                                    population_size=pop_size,
                                    number_of_generations=gen_cnt,
                                    selection=TournamentSelection(tour_size),
                                    crossover_probability=cross,
                                    mutation_rate=mut,
                                    elitism=1,
                                    terminate_on_stagnation_in_x_generations=stag,
                                    debug_stream=None)
        print(f"\r{j}\tGeneration Count: {i}", end="")
    